In [1]:
# Boilerplate setup for data standards:
from typing import Optional, Union, List, Dict, Any

from IrisBackendv3.codec.payload import TelemetryPayload
from IrisBackendv3.data_standards import DataStandards
from IrisBackendv3.codec.settings import set_codec_standards
from IrisBackendv3.data_standards.prebuilt import add_to_standards, watchdog_detailed_status_heartbeat, watchdog_heartbeat_tvac, watchdog_heartbeat, watchdog_command_response

import ulid
from IrisBackendv3.data_standards.logging import logger
logger.setLevel('SUCCESS')

standards = DataStandards.build_standards()
add_to_standards(standards, [
    watchdog_detailed_status_heartbeat,
    watchdog_heartbeat_tvac,
    watchdog_heartbeat,
    watchdog_command_response
])
set_codec_standards(standards)

2022-02-05 17:45:59 connors-mbp.wifi.local.cmu.edu IrisBackendv3.data_standards.logging[26211] ERROR Standards Formatting Exception while compiling component at '../FlightSoftware/fprime/Svc/Health/HealthComponentAi.xml'.
 This module is being skipped and will not be in GSW.
 Original `StandardsFormattingException`: Formatting of standards file '../FlightSoftware/fprime/Svc/Health/HealthComponentAi.xml' (or its expanded state) does not conform to expectations: No <instance> of <component> Health found in FPrime Topology for module defined at 3.


In [2]:
# Create dict with empty storage for lists of all values retrieved for every possible telemetry channel (to store the latest value):
storage: Dict[str, List[Any]] = dict((f"{m.name}_{ch.name}", []) for m in standards.modules.values() for ch in m.telemetry.values())

In [3]:
# A mock of the core FLEUR executive:
def fleur_look_for_errors() -> None:
    pass # do something based on storage

def fleur_handler(payload: TelemetryPayload) -> None:
    module_def = standards.modules[payload.module_id]
    channel_def = module_def.telemetry[payload.channel_id]
    
    module_name = module_def.name
    channel_name = channel_def.name
    storage_idx = f"{module_name}_{channel_name}"
    
    storage[storage_idx].append(payload.data) # I'd recommend a ring buffer instead of a list (push onto it) -- fixed length FIFO stack, collections.deque
    
    fleur_look_for_errors()

In [4]:
# Load a bunch of telemetry from a pcap:
from scripts.tiny_apps.parse_pcap import parse_pcap
import attr
@attr.s(auto_attribs=True)
class PcapParseOpts:
    pcap_file: str = './test-data/Iris_210503_18_hours_of_telem.pcap'
    port: Union[str,int] = 'any'
    cache_dir: str = './out/pcap_logs/'
    cache_prefix: str = f'iris_pcap_{ulid.new()}'
    cache_telem: bool = True
    cli_log: bool = False
    plot: bool = False
    packetgap: int = 0
    deadspace: int = 0
    log_level: str = 'INFO'

pcap_opts = PcapParseOpts()

pcap_data = parse_pcap(pcap_opts)
logger.notice("DONE LOADING!")

Parsing pcap file at ./test-data/Iris_210503_18_hours_of_telem.pcap, looking for UDP packets on port any.
Results will be cached to ./pcap_logs/, using prefix iris_pcap_01FV611MFV4KNS1QVP3JTCXTSB.
 Note: to see all configuration options, run in CLI with `--help` argument.
	 > Opening pcap . . .
	 > Found 37644 packets in pcap.
	 > Found 37593 matching packets (UDP with port any) in pcap.
	 > Loading any existing cache . . .
	 > Extracting packets (in silent mode) . . .
		 > Parsed     1/37593 packets.
		 > Parsed   101/37593 packets.
		 > Parsed   201/37593 packets.
		 > Parsed   301/37593 packets.
		 > Parsed   401/37593 packets.
		 > Parsed   501/37593 packets.
		 > Parsed   601/37593 packets.
		 > Parsed   701/37593 packets.
		 > Parsed   801/37593 packets.
		 > Parsed   901/37593 packets.
		 > Parsed  1001/37593 packets.
		 > Parsed  1101/37593 packets.
		 > Parsed  1201/37593 packets.
		 > Parsed  1301/37593 packets.
		 > Parsed  1401/37593 packets.
		 > Parsed  1501/37593 packets

#### Replaying a bunch of old data at a fixed rate into the FLEUR layer:

In [5]:
from parse_pcap import replay_telemetry_at_fixed_rate
import asyncio

time_between_packets = 2000 # [ms]
max_time_to_run = 60 # [s] timeout for application

async def replay():
    gen = replay_telemetry_at_fixed_rate(
        packets = pcap_data['extracted_packets'],
        fixed_rate = time_between_packets
    )
    try:
        async for payloads_in_packet in gen:
            for telemetry_payload in payloads_in_packet:
                fleur_handler(telemetry_payload)
            # Print all non-empty entries in storage:
            print(dict((k,v) for k,v in storage.items() if len(v) > 0))
    except asyncio.CancelledError:
        print('\t > Telemetry replay cancelled.')

async def main(timeout: int = max_time_to_run):
    try:
        await asyncio.wait_for(replay(), timeout = timeout)
    except asyncio.TimeoutError:
        print(f'Application timed out after {timeout:.3f}s.')

# asyncio.run(main()) # run this in standalone, can't run in jupyter
await main() # type: ignore # just run this in jupyter instead


{'WatchdogHeartbeatTvac_AdcTempRaw': [4095], 'WatchdogHeartbeatTvac_AdcTempKelvin': [260.8522058823529], 'WatchdogHeartbeatTvac_ChargeRaw': [0], 'WatchdogHeartbeatTvac_ChargeMah': [0], 'WatchdogHeartbeatTvac_VoltageRaw': [0], 'WatchdogHeartbeatTvac_Voltage': [0.0], 'WatchdogHeartbeatTvac_CurrentRaw': [0], 'WatchdogHeartbeatTvac_CurrentAmps': [-1.2799999990407], 'WatchdogHeartbeatTvac_FuelTempRaw': [0], 'WatchdogHeartbeatTvac_FuelTempKelvin': [0.0], 'WatchdogHeartbeatTvac_KpHeater': [500], 'WatchdogHeartbeatTvac_HeaterSetpoint': [3325], 'WatchdogHeartbeatTvac_HeaterSetpointKelvin': [273.56033434650453], 'WatchdogHeartbeatTvac_HeaterWindow': [60], 'WatchdogHeartbeatTvac_HeaterWindowKelvin': [0.9205281871881631], 'WatchdogHeartbeatTvac_HeaterPwmLimit': [8500], 'WatchdogHeartbeatTvac_WatchdogMode': [8], 'WatchdogHeartbeatTvac_HeaterStatus': [1], 'WatchdogHeartbeatTvac_HeatingControlEnabled': [1], 'WatchdogHeartbeatTvac_HeaterPwmDutyCycle': [8500]}
{'WatchdogHeartbeatTvac_AdcTempRaw': [4095

#### EXAMPLE JUST FEEDING A BUNCH OF FAKE TELEM INTO THE `fleur_handler`:

In [6]:
# Grab a list of all the available channels:
from random import randint
all_channels = [(m.ID, ch.ID) for m in standards.modules.values() for ch in m.telemetry.values()]

In [7]:
# Spit out fake data:
from datetime import datetime

start = datetime.now()
for _ in range(1000):
    random_channel = all_channels[randint(0, len(all_channels)-1)]
    fake_payload = TelemetryPayload(
        module_id = random_channel[0],
        channel_id = random_channel[1],
        timestamp = (datetime.now() - start).microseconds/1000,
        data = randint(0, 1000)
    )
    fleur_handler(fake_payload)

In [8]:
# Print the results:
storage

{'BlockDriver_BdCycles': [866, 556, 218, 63],
 'ActiveRateGroup-RateGroupLowFreq_RgMaxTime': [35000,
  36000,
  664000,
  667000,
  327,
  854,
  361,
  995,
  850,
  193,
  668],
 'ActiveRateGroup-RateGroupLowFreq_RgCycleSlips': [871,
  703,
  250,
  164,
  440,
  980,
  236,
  872],
 'ActiveRateGroup-RateGroupMedFreq_RgMaxTime': [2000,
  3000,
  564,
  659,
  236,
  471,
  985,
  853,
  917,
  60,
  720,
  283],
 'ActiveRateGroup-RateGroupMedFreq_RgCycleSlips': [878, 703, 373],
 'ActiveRateGroup-RateGroupHiFreq_RgMaxTime': [0, 1000, 177, 684, 963],
 'ActiveRateGroup-RateGroupHiFreq_RgCycleSlips': [810,
  624,
  239,
  553,
  274,
  867],
 'CommandDispatcher_CommandsDispatched': [767, 783, 358, 923, 944, 621],
 'CommandDispatcher_CommandErrors': [267, 198, 630, 272],
 'GroundInterface_GiUplinkSeqNum': [65, 864, 854, 878, 756, 443, 103, 237],
 'GroundInterface_GiDownlinkSeqNum': [274, 258, 272, 304, 83],
 'GroundInterface_GiPacketsReceived': [954, 58, 10, 951, 216, 2, 720, 573],
 'Grou

In [9]:
# Non-working example of roughly how `ipc` should integrate:
"""
from IrisBackendv3 import ipc 
context = ipc.create_context()
socket = ipc.create_socket(context, ipc.SocketType.SUBSCRIBER, ipc.Port.TELEM_WINDOW)
def run() -> None:
    ipc_payload = ipc.read_from(socket)
    message = ipc_payload.msg
    handler(message)
    
while True:
    run()
"""

'\nfrom IrisBackendv3 import ipc \ncontext = ipc.create_context()\nsocket = ipc.create_socket(context, ipc.SocketType.SUBSCRIBER, ipc.Port.TELEM_WINDOW)\ndef run() -> None:\n    ipc_payload = ipc.read_from(socket)\n    message = ipc_payload.msg\n    handler(message)\n    \nwhile True:\n    run()\n'